In [1]:
import pandas as pd
import numpy as np

In [2]:
#np.set_printoptions(threshold=np.nan)

In [3]:
files=[]
import os
for file in os.listdir("./DataSet"):
    if file.endswith(".csv"):
        files.append(file)
files.sort()
print(files)
    

['Participant_1.csv', 'Participant_10.csv', 'Participant_2.csv', 'Participant_3.csv', 'Participant_4.csv', 'Participant_5.csv', 'Participant_6.csv', 'Participant_7.csv', 'Participant_8.csv', 'Participant_9.csv']


# FORMATTING THE DATA

In [4]:
mapping = {'walking': 0, 'standing': 1, 'jogging': 2, 'sitting': 3, 'biking': 4, 'upstairs': 5, 'downstairs': 6}

training_data=None
training_output=None
testing_data=None
testing_output=None

ratio=7

count=0
for file in files:
    count+=1
    df = pd.read_csv('./DataSet/'+file, index_col=False, header=1);  
    #print(df.describe())
    print(file)
    data=None
    for i in range(1,6):
        dff=df.ix[:,14*i-13:14*i-1]
        dff['activity'] = df['Activity']
        
        dff=dff.replace({'activity': mapping})
        
        if data is None:
            data=dff.as_matrix()
        else:
            data=np.append(data,dff.as_matrix(),axis=0)

    print(data.shape)
    #print(data)
    
    linear_data_x=None
    linear_data_y=None
    window_size=100
    window_shift=10
    label = 0
    index=0
    data_flag = False
    
    
    print("Clubbing data")
    while data_flag!=True:
        while data[index+window_size -1][12] == label and data_flag!=True:
            if linear_data_x is None:
                linear_data_x = data[i:i+window_size,:12].ravel()
                linear_data_y = label
            else:
                linear_data_x = np.vstack((linear_data_x,data[i:i+window_size,:12].ravel()))
                linear_data_y = np.hstack((linear_data_y,label))
            
            index+=window_shift
            
            if index>= data.shape[0] or index+window_size>=data.shape[0]:
                data_flag=True
                break
        
        label+=1
        print(index,label)
        #print(label)
        
        if label == 7:
            label=0
        
        else:
            #print(label,data[index][12])
            while data[index][12] != label:
                index+=1
                #if index%1000 == 0:
                    #print(index,data[index][12],label)
    
    #print(count)
    if count<=ratio:
        if training_data is None:
            training_data=linear_data_x
            training_output=linear_data_y
        else:
            training_data=np.vstack((training_data,linear_data_x))
            training_output=np.append(training_output,linear_data_y)
    else:
        if testing_data is None:
            testing_data=linear_data_x
            testing_output=linear_data_y
        else:
            testing_data=np.vstack((testing_data,linear_data_x))
            testing_output=np.append(testing_output,linear_data_y)

    print(training_data.shape,training_output.shape)
    if testing_data is not None:
        print(testing_data.shape,testing_output.shape)


Participant_1.csv
(315000, 13)
Clubbing data
8910 1
17910 2
26910 3
35910 4
44910 5
53910 6
62910 7
71910 1
80910 2
89910 3
98910 4
107910 5
116910 6
125910 7
134910 1
143910 2
152910 3
161910 4
170910 5
179910 6
188910 7
197910 1
206910 2
215910 3
224910 4
233910 5
242910 6
251910 7
260910 1
269910 2
278910 3
287910 4
296910 5
305910 6
314900 7
(31220, 1200) (31220,)
Participant_10.csv
(315000, 13)
Clubbing data
8910 1
17910 2
26910 3
35910 4
44910 5
53910 6
62910 7
71910 1
80910 2
89910 3
98910 4
107910 5
116910 6
125910 7
134910 1
143910 2
152910 3
161910 4
170910 5
179910 6
188910 7
197910 1
206910 2
215910 3
224910 4
233910 5
242910 6
251910 7
260910 1
269910 2
278910 3
287910 4
296910 5
305910 6
314900 7
(62440, 1200) (62440,)
Participant_2.csv
(315000, 13)
Clubbing data
8910 1
17910 2
26910 3
35910 4
44910 5
53910 6
62910 7
71910 1
80910 2
89910 3
98910 4
107910 5
116910 6
125910 7
134910 1
143910 2
152910 3
161910 4
170910 5
179910 6
188910 7
197910 1
206910 2
215910 3
224910 4

In [5]:
import h5py
h5f = h5py.File('preprocessed_data_winSize100_winShift10.h5', 'w')
h5f.create_dataset('training_data', data=training_data)
h5f.create_dataset('testing_data', data=testing_data)
h5f.create_dataset('training_output', data=training_output)
h5f.create_dataset('testing_output', data=testing_output)

<HDF5 dataset "testing_output": shape (93660,), type "<i8">

In [6]:
h5f.close()

In [7]:
h5f = h5py.File('preprocessed_data_winSize100_winShift10.h5','r')
b = h5f['testing_data'][:]
print(b.shape)

(93660, 1200)


In [ ]:
print(np.mean(training_data,axis=0),np.std(training_data,axis=0))
print(np.mean(testing_data,axis=0),np.std(testing_data,axis=0))